In [1]:
import pandas as pd
import geopandas as gpd
import re
import folium
from shapely.geometry import Point
from shapely import wkt


In [4]:
data = pd.read_csv("/Users/yu/Desktop/Räumliche Analyse/query.tsv", sep = "\t" , encoding = "utf-8")

In [5]:
data.columns = ["Straßenname", "Personname", "Gender", "Beruf", "coordinate", "PLZ"]

In [6]:
# Funktion zum Bereinigen und Formatieren der Koordinaten
def clean_coordinate(coord):
    match = re.search(r'\((.*?)\)', coord)
    if match:
        coord = match.group(1)
    return 'POINT ({})'.format(coord)

In [7]:
# Bereinigung und Konvertierung der Koordinaten in die "geometry"-Spalte
data['coordinate'] = data['coordinate'].apply(lambda coord: clean_coordinate(coord))
data['coordinate'] = data['coordinate'].apply(wkt.loads)

In [8]:
# Erstellen des GeoDataFrames
gdf = gpd.GeoDataFrame(data, geometry='coordinate', crs=4326)

In [9]:
#sortieren nach weibliche Straßenname und löchen alle verdoppelte Namen
name_w = data[data["Gender"].str.contains("weiblich", na=False)]
name_w = name_w.drop_duplicates(subset='Straßenname')

In [10]:
name = name_w["Straßenname"].tolist()

In [11]:
len(name)

97

In [12]:
# Erstellen des GeoDataFrames
name_w = gpd.GeoDataFrame(name_w, geometry='coordinate', crs=4326)

In [15]:
# Anzeigen der weiblichen Straßennamen auf der Karte
map_w = name_w.explore(
    color="red",
    marker_kwds=dict(radius=3),
    popup=True,
    width=800,
    height=500
)

ImportError: The 'folium', 'matplotlib' and 'mapclassify' packages are required for 'explore()'. You can install them using 'conda install -c conda-forge folium matplotlib mapclassify' or 'pip install folium matplotlib mapclassify'.

In [14]:
#sortieren nach männlichen Straßenname und löchen alle verdoppelte Namen
name_m = data[data["Gender"].str.contains("männlich", na=False)]
name_m = name_m.drop_duplicates(subset='Straßenname')

In [11]:
# Erstellen des GeoDataFrames
name_m = gpd.GeoDataFrame(name_m, geometry='coordinate', crs=4326)

In [12]:
# Anzeigen der männlichen Straßennamen auf der Karte
map_m = name_m.explore(
    color="cornflowerblue",
    marker_kwds=dict(radius=2),
    popup=True,
    width=800,
    height=500
)

In [13]:
#weibliche und männliche Straßen zusammen darstellen
map = name_m.explore(
    color = "cornflowerblue", 
    marker_kwds=dict(radius=2), 
    popup =True,
    width =800,
    height = 500
    )
name_w.explore(
    m = map, 
    color = "red", 
    marker_kwds=dict(radius=2), 
    popup =True
    )

Stadtbezirke auf der Karte darstellen

In [14]:
stadtbezirk = pd.read_csv("../Datenkompetenz_Räumliche_Analyse/Daten/Postleitzahlgebiete.csv", sep = ";", encoding='utf-8')
stadtbezirk.head()

,autoid,geom,objectid,plz,lage,text
0,1,SRID=4326;POLYGON((13.95300474041579086304 51....,22,1328,Stadt,01328 Dresden
1,2,SRID=4326;POLYGON((13.81127643829515427853 50....,23,1259,Stadt,01259 Dresden
2,3,SRID=4326;POLYGON((13.81127643829515427853 50....,24,1257,Stadt,01257 Dresden
3,4,SRID=4326;POLYGON((13.78965781765939802028 50....,25,1239,Stadt,01239 Dresden
4,5,SRID=4326;POLYGON((13.84400120203005890573 51....,26,1279,Stadt,01279 Dresden


In [15]:
#die Werte in der Spalte geom trennen
stadtbezirk[["SRID", "geometry"]] = stadtbezirk["geom"].str.split(';', n=1, expand=True)

In [16]:
# Spalte "geometry" in Geometrieobjekte konvertieren
stadtbezirk['geometry'] = stadtbezirk['geometry'].apply(wkt.loads)

# GeoDataFrame erstellen
gdf = gpd.GeoDataFrame(stadtbezirk, geometry='geometry')

In [18]:
stadtbezirk.head()

,autoid,geom,objectid,plz,lage,text,SRID,geometry
0,1,SRID=4326;POLYGON((13.95300474041579086304 51....,22,1328,Stadt,01328 Dresden,SRID=4326,POLYGON ((13.95300474041579 51.073935922095245...
1,2,SRID=4326;POLYGON((13.81127643829515427853 50....,23,1259,Stadt,01259 Dresden,SRID=4326,POLYGON ((13.811276438295154 50.97701431281828...
2,3,SRID=4326;POLYGON((13.81127643829515427853 50....,24,1257,Stadt,01257 Dresden,SRID=4326,POLYGON ((13.811276438295154 50.97701431281828...
3,4,SRID=4326;POLYGON((13.78965781765939802028 50....,25,1239,Stadt,01239 Dresden,SRID=4326,POLYGON ((13.789657817659398 50.98449477826835...
4,5,SRID=4326;POLYGON((13.84400120203005890573 51....,26,1279,Stadt,01279 Dresden,SRID=4326,POLYGON ((13.844001202030059 51.01500064434166...


In [19]:
#sortieren nach PLZ (weiblich) und die erste 5 Werte als DataFrame speichern
top_name_w = name_w["PLZ"].value_counts().head(5)
top_name_w = pd.DataFrame({"PLZ": top_name_w.index, "Top 5 w": top_name_w.values})
top_name_w

,PLZ,Top 5 w
0,1099,11
1,1097,7
2,1069,6
3,1109,6
4,1219,6


In [20]:
#sortieren nach PLZ (männlich) und die erste 5 Werte als DataFrame speichern
top_name_m = name_m["PLZ"].value_counts().head(5)
top_name_m = pd.DataFrame({"PLZ": top_name_m.index, "Top 5 m": top_name_m.values})
top_name_m

,PLZ,Top 5 m
0,1219,71
1,1069,62
2,1326,45
3,1139,44
4,1307,44


In [21]:
# Gewünschte PLZs
desired_plzs = [1219, 1069, 1326, 1139, 1307]

# Filtern der Tabelle basierend auf den gewünschten PLZs
filtered_rows = stadtbezirk[stadtbezirk['plz'].isin(desired_plzs)]

# Extrahieren der Werte aus der Spalte "geometry"
desired_geometries = filtered_rows['geometry']


In [22]:
#extraxierte Werte als Data.Frame speichern
plz_m = pd.DataFrame({"PLZ": desired_plzs, "geometry": desired_geometries})

In [23]:
#zwei Tabellen zusammenfügen 
plz_m = pd.merge(top_name_m, plz_m, on='PLZ', how='inner')

In [24]:
plz_m.head()

,PLZ,Top 5 m,geometry
0,1219,71,POLYGON ((13.755673600646244 51.04565992705217...
1,1069,62,POLYGON ((13.758163496158168 51.04424475640583...
2,1326,45,POLYGON ((13.810637733296721 51.06367250402122...
3,1139,44,POLYGON ((13.755818066215989 51.05494867289798...
4,1307,44,"POLYGON ((13.661201087025251 51.0901364905306,..."


In [25]:
# Gewünschte PLZs
desired_plzs = [1219, 1069, 1326, 1139, 1307]

In [26]:
# Filtern der Tabelle basierend auf den gewünschten PLZs
filtered_rows = stadtbezirk[stadtbezirk['plz'].isin(desired_plzs)]

In [27]:
# GeoDataFrame mit den gefilterten Bezirken erstellen
filtered_gdf = gpd.GeoDataFrame(filtered_rows, geometry='geometry')

In [28]:
# Mittelpunkt der gefilterten Bezirke berechnen
centroid = filtered_gdf.unary_union.centroid

In [29]:
# Karte erstellen
m_m = folium.Map(location=[centroid.y, centroid.x], zoom_start=11.3)

In [30]:
# Polygone der gefilterten Bezirke zur Karte hinzufügen
for _, row in filtered_gdf.iterrows():
    folium.GeoJson(row['geometry']).add_to(m_m)

In [31]:
# Karte anzeigen
m_m

In [32]:
# Gewünschte PLZs
desired_plzs = [1099, 1097, 1069, 1109, 1219]

In [33]:
# Filtern der Tabelle basierend auf den gewünschten PLZs
filtered_rows = stadtbezirk[stadtbezirk["plz"].isin(desired_plzs)]

In [34]:
# GeoDataFrame mit den gefilterten Bezirken erstellen
#filtered_gdf = gpd.GeoDataFrame(filtered_rows, geometry='geometry')

In [35]:
# Mittelpunkt der gefilterten Bezirke berechnen
centroid = filtered_gdf.unary_union.centroid

In [38]:
# Karte erstellen
m_w = folium.Map(location=[centroid.y, centroid.x], zoom_start=11.3)

In [39]:
# Funktion zum Festlegen des Polygonstils
def style_function(feature):
    return {
        'fillColor': 'red',
        'color': 'red',
        'weight': 1.5,
        'fillOpacity': 0.7
    }

In [40]:
# Polygone der gefilterten Bezirke zur Karte hinzufügen
for _, row in filtered_gdf.iterrows():
    folium.GeoJson(row['geometry'], style_function=style_function).add_to(m_w)

In [41]:

# Karte anzeigen
m_w


Alle Karte auf dem Computer speichern

In [42]:
m_w.save("../Datenkompetenz_Räumliche_Analyse/Karten/top_5_w.html")

In [43]:
m_m.save("../Datenkompetenz_Räumliche_Analyse/Karten/top_5_m.html")

In [44]:
map.save("../Datenkompetenz_Räumliche_Analyse/Karten/Strasse_m_und_w.html")

In [45]:
map_m.save("../Datenkompetenz_Räumliche_Analyse/Karten/Strasse_m.html")

In [46]:
map_w.save("../Datenkompetenz_Räumliche_Analyse/Karten/Strasse_w.html")